## SSRO Demo 
## Preprocessing Data Input Tables Site Specific RShiny Demo App
Notes:
- asdf

In [1]:
# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd # the library that lets us read in shapefiles
import geoplot as gplt # for plotting maps

# visulizaiton
import seaborn as sns
import matplotlib.pyplot as plt

# Cleanup
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
# Setting work directory
cwd = os.getcwd()
print("current directory = " + cwd)

os.chdir("../NotEssentialData")
print("new directory = " + os.path.abspath(os.curdir))

current directory = C:\Users\rjame\Documents\WSWC Documents\WaDE Side Projects Local\20221017 Rshiny SS Demo\SSRO_Demo\bonus code
new directory = C:\Users\rjame\Documents\WSWC Documents\WaDE Side Projects Local\20221017 Rshiny SS Demo\SSRO_Demo\NotEssentialData


## Inputs

In [3]:
#Sites Input
fileInput = "Sites.csv"
dfs = pd.read_csv(fileInput)
print(len(dfs))
dfs.head(1)

11044


,SiteUUID,SiteName,SiteNativeID,SiteTypeCV,WaDENameS,Longitude,Latitude,PODorPOUSite,WaDENameWS,TimeStep,VariableCV,WaDENameV,VariableSpecificCV,HasRecords
0,COssro_S1011,RIO SOUTH WELL NO 2,1605061,WELL,Well / Pump / Spring / Groundwater Point,-104.858927,37.633941,Observation Site,Groundwater,Annual,Discharge Flow,Discharge Flow,Discharge Flow_Annual_DivTotal_Groundwater,Yes


## Clean Sites Data

In [4]:
# Clean input.  Remove no value sites

dfs = dfs.drop_duplicates() # Drop Duplicate Rows
dfs = dfs.dropna(subset=['VariableSpecificCV']).reset_index(drop=True) # drop no value sites
dfs = dfs.replace(np.nan, "")  # Replaces NaN values with blank.
print(len(dfs))
dfs.head(1)

10915


,SiteUUID,SiteName,SiteNativeID,SiteTypeCV,WaDENameS,Longitude,Latitude,PODorPOUSite,WaDENameWS,TimeStep,VariableCV,WaDENameV,VariableSpecificCV,HasRecords
0,COssro_S1011,RIO SOUTH WELL NO 2,1605061,WELL,Well / Pump / Spring / Groundwater Point,-104.858927,37.633941,Observation Site,Groundwater,Annual,Discharge Flow,Discharge Flow,Discharge Flow_Annual_DivTotal_Groundwater,Yes


In [5]:
for x in dfs['VariableCV'].sort_values().unique():
    print(f'"' + x + '",')

"Discharge Flow",
"Reservoir Level",


In [6]:
# Fixing blank WaDENameS
def updateWaDENameS(Val):
    Val = str(Val).strip()
    if Val == "" or pd.isnull(Val) or Val == "":
        outString = "Unspecified"
    else:
        outString = Val
    return outString

dfs['WaDENameS'] = dfs.apply(lambda row: updateWaDENameS(row['WaDENameS']), axis=1)
for x in dfs['WaDENameS'].sort_values().unique():
    print(f'"' + x + '",')

"Canal / Ditch / Diversion",
"Reservoir",
"Stream Gage",
"Surface Water Point",
"Unspecified",
"Well / Pump / Spring / Groundwater Point",


In [7]:
# Fixing blank WaDENameWS
def updateWaDENameWS(Val):
    Val = str(Val).strip()
    if Val == "" or pd.isnull(Val) or Val == "":
        outString = "Unspecified"
    else:
        outString = Val
    return outString

dfs['WaDENameWS'] = dfs.apply(lambda row: updateWaDENameWS(row['WaDENameWS']), axis=1)
for x in dfs['WaDENameWS'].sort_values().unique():
    print(f'"' + x + '",')

"Groundwater",
"Surface Water",
"Unspecified",


In [8]:
for x in dfs['TimeStep'].sort_values().unique():
    print(f'"' + x + '",')

"Annual",
"Daily",
"Monthly",


In [9]:
# add State field

def createState(Val):
    Val = Val.strip()
    outString = Val[:2]
    return outString

dfs['State'] = dfs.apply(lambda row: createState(row['SiteUUID']), axis=1)
for x in dfs['State'].sort_values().unique():
    print(f'"' + x + '",')

"CA",
"CO",
"ID",
"MT",
"NE",
"NV",
"OR",


In [10]:
# Groupby SiteUUID
dfs = dfs.groupby(['SiteUUID']).agg(lambda x: ", ".join([str(elem) for elem in (list(set(x))) if elem!=""])).replace(np.nan, "").reset_index()
print(len(dfs))
dfs.head(1)

KeyboardInterrupt: 

In [ ]:
# count num of VariableSpecificCV after groupby

def countNumVar(Val):
    Val = list(Val.split(","))
    if Val == [""]:
        ValLen = 0
    else:
        ValLen = len(Val)
    return ValLen

dfs['CountVar'] = dfs.apply(lambda row: countNumVar(row['VariableSpecificCV']), axis=1)
for x in dfs['CountVar'].sort_values().unique():
    print(f'"' + str(x) + '",')

In [ ]:
# attach timeframe based on SiteUUID

fileInput = "start_end_Sites.csv"
dfstf = pd.read_csv(fileInput)
dfs = pd.merge(dfs, dfstf, on='SiteUUID', how='left')
print(len(dfs))
dfs.head(1)

In [ ]:
# Temp Fix

# remove lat & long entries that contain a ','
dfs = dfs[dfs['Latitude'].str.contains(',') == False]
dfs = dfs[dfs['Longitude'].str.contains(',') == False]

# Ensure long is < 0 and > -125 (few ID sites out of bounds)
# Ensure lat is > 10 and < 50 (few MT sites out of bounds).
dfs['Latitude'] = pd.to_numeric(dfs['Latitude'], errors='coerce')
dfs['Longitude'] = pd.to_numeric(dfs['Longitude'], errors='coerce')
dfs = dfs[(dfs['Latitude'].astype(int) > 10) & (dfs['Latitude'].astype(int) < 50)]
dfs = dfs[(dfs['Longitude'].astype(int) < 0) & (dfs['Longitude'].astype(int) > -125)]

print(len(dfs))
dfs.head(1)

In [ ]:
#Inspect output
dfs

In [ ]:
# Exporting output files.
dfs.to_excel('Sites_v2.xlsx', index=False)

## Mapping to Validate Sites

In [ ]:
# map points
# convert dataframe to geodataframe
gdf_sites = gpd.GeoDataFrame(dfs, geometry=gpd.points_from_xy(dfs.Longitude.astype(float), dfs.Latitude.astype(float)), crs="EPSG:4326")
gplt.pointplot(gdf_sites)